In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation, rc
from IPython.display import HTML

import numpy as np
import matplotlib.pyplot as plt
import sys
import torch

sys.path.append('../code')

import toy_data as toy
from vae import VAE

from sklearn import gaussian_process

In [ ]:
N = 2500
batch_size = 32

# introduce time dependency
rbf = gaussian_process.kernels.RBF(length_scale=2)
GP = gaussian_process.GaussianProcessRegressor(kernel=rbf)

t = np.linspace(0, 120, N)
y = np.empty((N, 3))
y[:, 0] = GP.sample_y(t[:, None], random_state=None)[:, 0]
y[:, 1] = GP.sample_y(t[:, None], random_state=None)[:, 0]
y[:, 2] = GP.sample_y(t[:, None], random_state=None)[:, 0]
labels = y

plt.plot(t, labels)

In [ ]:
a, b, c = (np.random.rand(3)+1)/2
imgs = []
for label in labels:
    coords = toy.forward(label, [a, b, c])
    imgs += [toy.keypoint_to_image(coords)]

In [ ]:
fig, ax = plt.subplots()
img = imgs[0]
mimg = plt.imshow(img)

def init():
    mimg.set_data(img)
    return (mimg,)

def animate(i):
    img = imgs[2*i]
    mimg.set_data(img)
    return (mimg,)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=len(imgs)//2, interval=25, 
                               blit=True)

# HTML(anim.to_html5_video())

In [ ]:
def plot_mean_variance_imgs():
    average = np.zeros((28, 28))
    dev = np.zeros((28, 28))
    labels = 2*np.pi*(np.random.rand(N, 2)-0.5)
    batch_generator = list(toy.make_batch_generator(labels, [a,b], batch_size))
    for i, (batch_img, _ ) in enumerate(batch_generator):
        average = average + np.sum(batch_img[:, 0], axis=0)
        dev = dev + np.sum(batch_img[:, 0]**2, axis=0)
    average = average/i
    dev = dev/i

    fig, ax = plt.subplots(ncols=2)
    ax[0].imshow(average)
    ax[1].imshow(average**2-dev)

In [ ]:
# Independent across time
a, b = (np.random.rand(2)+1)/2
print(a, b)
labels = 2*np.pi*(np.random.rand(N, 2)-0.5)

batch_generator = toy.make_batch_generator(labels, [a,b], batch_size)

device = torch.device("cpu")
bottleneck = 10
model = VAE(bottleneck=bottleneck).to(device)
train_loss = model.fit(batch_generator, max_iter=200)

In [ ]:
plt.plot(train_loss)

In [ ]:
with torch.no_grad():
    sample = torch.randn(9, bottleneck).to(device)
    sample = model.decode(sample).cpu()

In [ ]:
from itertools import product
to_ind = np.array(list(product(range(3), range(3))))

fig, ax = plt.subplots(3, 3, sharex=True, sharey=True)
fig.set_size_inches((10, 10))
for i, img in enumerate(np.array(sample)):
    ind = to_ind[i]
    ax[ind[0], ind[1]].imshow(img.reshape(28, 28))
plt.tight_layout()
plt.show()